import the data

In [585]:
import pandas as pd

In [586]:
df = pd.read_csv("result_testdata.csv", delimiter=",")
df

,x,y,z,f
0,0.876300,0.087160,0.678294,1.741426
1,0.506543,0.427771,0.117943,1.028571
2,0.469181,0.208947,0.862431,1.539668
3,0.997632,0.038999,0.628393,1.821145
4,0.680144,0.432733,0.571025,1.691445
...,...,...,...,...
569,0.213704,0.183989,0.577722,1.052330
570,0.727660,0.021602,0.820556,1.497021
571,0.452470,0.848223,0.896740,2.022459
572,0.470553,0.443268,0.204643,1.091847


build the network

https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/

In [587]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import autograd
import math
from sklearn.metrics import mean_squared_error

In [588]:
X = df.iloc[:,:3]
y = df.iloc[:,3]
# y = df.iloc[:,:3].sum(axis=1) 

In [589]:
X = torch.tensor(X.values.astype(np.float32), requires_grad=True)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [590]:
X.shape

torch.Size([574, 3])

In [591]:
y.shape

torch.Size([574, 1])

In [592]:
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(3, 160)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(160, 80)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(80, 16)
        self.act_output =  nn.Linear(16, 1)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
 
model = PimaClassifier()
print(model)

PimaClassifier(
  (hidden1): Linear(in_features=3, out_features=160, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=160, out_features=80, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=80, out_features=16, bias=True)
  (act_output): Linear(in_features=16, out_features=1, bias=True)
)


In [593]:
y

tensor([[1.7414],
        [1.0286],
        [1.5397],
        [1.8211],
        [1.6914],
        [1.4429],
        [1.6620],
        [1.2481],
        [0.7386],
        [1.8292],
        [2.2020],
        [1.5843],
        [2.3136],
        [1.1235],
        [1.4134],
        [1.4076],
        [0.6883],
        [2.5799],
        [1.4953],
        [1.5863],
        [1.2879],
        [1.1798],
        [2.1594],
        [1.7896],
        [1.9594],
        [1.9774],
        [1.6865],
        [2.3089],
        [1.2882],
        [1.0681],
        [1.2656],
        [1.2819],
        [1.3000],
        [1.1343],
        [2.1116],
        [0.8379],
        [0.5024],
        [1.1742],
        [2.2438],
        [0.7275],
        [1.0485],
        [1.3241],
        [2.0168],
        [1.8547],
        [0.9294],
        [2.1854],
        [0.8303],
        [0.9071],
        [0.7578],
        [1.0846],
        [1.8044],
        [0.7077],
        [0.5011],
        [2.0659],
        [1.9117],
        [1

In [594]:
loss_fn = nn.MSELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size].clone()
        Xbatch.requires_grad=True
        y_pred = model(Xbatch)
        f_x_t = autograd.grad(y_pred,Xbatch,torch.ones([Xbatch.shape[0], 1]), retain_graph=True, create_graph=True)[0] #first derivative
        print(f_x_t)
        ybatch = y[i:i+batch_size]
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.03893100470304489
Finished epoch 1, latest loss 0.00971320178359747
Finished epoch 2, latest loss 0.005221494939178228
Finished epoch 3, latest loss 0.004688470624387264
Finished epoch 4, latest loss 0.0041638631373643875
Finished epoch 5, latest loss 0.003627225523814559
Finished epoch 6, latest loss 0.0026280786842107773
Finished epoch 7, latest loss 0.001826498773880303
Finished epoch 8, latest loss 0.0013636208605021238
Finished epoch 9, latest loss 0.0010666153393685818


In [595]:
# compute accuracy (no_grad is optional)
# with torch.no_grad():
#     y_pred = model(X)

X.requires_grad_()
X.retain_grad()
y_pred = model(X)

trainScore = math.sqrt(mean_squared_error(y_pred.detach().numpy(),y.detach().numpy()))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 0.02 RMSE


In [596]:
X[0]

tensor([0.8763, 0.0872, 0.6783], grad_fn=<SelectBackward0>)

In [597]:
y[0]

tensor([1.7414])

In [598]:
model(X[0])

tensor([1.7134], grad_fn=<AddBackward0>)

In [599]:
X.grad

tensor([[-0.0367, -0.0537, -0.0499],
        [ 0.0605,  0.0329,  0.0393],
        [ 0.0316,  0.0133,  0.0243],
        ...,
        [ 0.0196,  0.0099,  0.0098],
        [ 0.1228,  0.0827,  0.1127],
        [ 0.0626,  0.0519,  0.0711]])

In [600]:
loss.grad

tensor(1.)

In [601]:
print(model(X).backward())

RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:
print(y.grad_fn)

None


In [ ]:
grads = autograd.grad(outputs=y, inputs=X, torch.ones([X.shape[0], 1]),retain_graph=True, create_graph=True)

RuntimeError: grad can be implicitly created only for scalar outputs